# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
#if not os.path.exists(COCO_MODEL_PATH):
#    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

Using TensorFlow backend.


## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [2]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

## Create Model and Load Trained Weights

In [3]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [4]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['background', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

## Run Object Detection

In [5]:
from skimage.transform import rescale, resize
import matplotlib.pyplot as plt

import json

In [6]:
images_json_file = r'C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\coco\images\image_info_test-dev2017.json'
categories_json_file = r'C:\Users\andor.neo\Desktop\casey\ml_ai\panopticapi-master\panoptic_coco_categories.json'
panoptic_json_file = r'C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\coco\annotations\panoptic_test2017.json'

In [7]:
with open(images_json_file, 'r') as f:
    images_d = json.load(f)
    
with open(categories_json_file, 'r') as f:
    categories_list = json.load(f)
categories = {el['id']: el for el in categories_list}
name2id = {el['name']: el['id'] for el in categories_list}

In [8]:
#Open specific image
image_dir = r'C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\coco\images\test\\'
imagelist = []
for n in range(len(images_d["images"])):
    image_file = image_dir + images_d['images'][n]['file_name']
    imagelist.append(image_file)
len(imagelist)

20288

In [9]:
def create_segments_info(r, png_zeros):
    
    png_img = png_zeros.copy()
    
    segments_info_list = []

    for n in range(len(r['class_ids'])):
        panoptic_ann = {}
        
        object_name = class_names[r['class_ids'][0]]
        category_id = name2id[object_name]
        
        mask = r["masks"][:, :, n]
        
        
        segment_id = id_generator.get_id(category_id)
        panoptic_ann["id"] = int(segment_id)
        panoptic_ann["category_id"] = int(category_id)
        panoptic_ann["iscrowd"] = 0
        panoptic_ann["bbox"] = [int(el) for el in r['rois'][n]]
        panoptic_ann["area"] = int(np.sum(mask))
        segments_info_list.append(panoptic_ann)
        
        rgb = id2rgb(segment_id)
        
        for n in range(3):
            
            dim = png_img[:, :, n]
            dim[mask] = int(rgb[n])

    return segments_info_list, png_img

In [10]:
class IdGenerator():
    def __init__(self, categories):
        self.taken_colors = set([0, 0, 0])
        self.categories = categories
        for category in self.categories.values():
            if category['isthing'] == 0:
                self.taken_colors.add(tuple(category['color']))

    def get_color(self, cat_id):
        def random_color(base, max_dist=30):
            new_color = base + np.random.randint(low=-max_dist,
                                                 high=max_dist+1,
                                                 size=3)
            return tuple(np.maximum(0, np.minimum(255, new_color)))

        category = self.categories[cat_id]
        if category['isthing'] == 0:
            return category['color']
        base_color_array = category['color']
        base_color = tuple(base_color_array)
        if base_color not in self.taken_colors:
            self.taken_colors.add(base_color)
            return base_color
        else:
            while True:
                color = random_color(base_color_array)
                if color not in self.taken_colors:
                     self.taken_colors.add(color)
                     return color

    def get_id(self, cat_id):
        color = self.get_color(cat_id)
        return rgb2id(color)

    def get_id_and_color(self, cat_id):
        color = self.get_color(cat_id)
        return rgb2id(color), color
    
def rgb2id(color):
    if isinstance(color, np.ndarray) and len(color.shape) == 3:
        if color.dtype == np.uint8:
            color = color.astype(np.uint32)
        return color[:, :, 0] + 256 * color[:, :, 1] + 256 * 256 * color[:, :, 2]
    return color[0] + 256 * color[1] + 256 * 256 * color[2]


def id2rgb(id_map):
    if isinstance(id_map, np.ndarray):
        id_map_copy = id_map.copy()
        rgb_shape = tuple(list(id_map.shape) + [3])
        rgb_map = np.zeros(rgb_shape, dtype=np.uint8)
        for i in range(3):
            rgb_map[..., i] = id_map_copy % 256
            id_map_copy //= 256
        return rgb_map
    color = []
    for i in range(3):
        color.append(id_map % 256)
        id_map //= 256
    return color

In [11]:
def create_exception_r(png_zeros):
    r = {}
    r['rois'] = [[0,0,0,0]]
    r['class_ids'] = [1]
    r['scores'] = [0]
    r['masks'] = np.array(png_zeros, dtype=bool)
    return r

In [12]:
import warnings 
warnings.filterwarnings(action='ignore')

In [135]:
import pickle

id_generator = IdGenerator(categories)

panoptic_json = images_d.copy()

panoptic_json_anns = []
annotation = {}

#Save sublists to later compile into master list
panoptic_json_dir = r'C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\coco\annotations\\'
sublist_file = 'pan_list_last.pkl'

current_extent = 3832

for n, idx in enumerate(imagelist[current_extent:3833]):
    n = n + current_extent
    file_name = panoptic_json['images'][n]['id']
    if n % 250 == 0:
        file_ext = r".pkl"
        with open(panoptic_json_dir + "_" + str(n) + "_" + file_ext, "wb") as fp:   #Pickling
        	pickle.dump(panoptic_json_anns, fp)
    
    if n % 1000 == 0:
        print("Processing image", n)
        print("File located at", idx)
    
    image = skimage.io.imread(idx)
    png_zeros = np.zeros((image.shape[0], image.shape[1], 3), dtype=int)
    
    # Run detection
    try:
        results = model.detect([image], verbose=0)
        
    except Exception as error:
        print('Found error at n =', n)
        print(error)
        r = create_exception_r(png_zeros)
        
        segments_info, png_img = create_segments_info(r, png_zeros)
        
        annotation['segments_info'] = segments_info
        panoptic_json_anns.append(annotation.copy())

        
        skimage.io.imsave(r'C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\coco\annotations\panoptic_annotations_test2017\\' + str(file_name).zfill(12) + '.png', png_zeros, vmin=0, vmax=255)
        skimage.io.imsave(r'C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\coco\annotations\panoptic_annotations_test2017_id\\' + str(file_name) + '.png', png_zeros, vmin=0, vmax=255)

        print("Created void entry due to B and W image number", n)
        
        continue

    #except Exception as error:
    #    print("A new exception occurred. voiding image")
    #    print(str(error))
    #    bw_r = create_exception_r(n)
    #    results = [bw_r]
    
    
    r = results[0]

    segments_info, png_img = create_segments_info(r, png_zeros)

    annotation['segments_info'] = segments_info
    panoptic_json_anns.append(annotation.copy())

    
    skimage.io.imsave(r'C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\coco\annotations\panoptic_annotations_test2017\\' + str(file_name).zfill(12) + '.png', png_img, vmin=0, vmax=255)
    skimage.io.imsave(r'C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\coco\annotations\panoptic_annotations_test2017_id\\' + str(file_name) + '.png', png_img, vmin=0, vmax=255)

    
with open(panoptic_json_dir + "3832.pkl", "wb") as fp:
    pickle.dump(panoptic_json_anns, fp)

    
#panoptic_json["annotations"] = panoptic_json_anns

#with open(panoptic_json_file, 'w') as f:
#    json.dump(panoptic_json, f)

In [16]:
import pickle
pickle_list = ["0_3832.pkl", "3833_8000.pkl", "8000_12000.pkl", "12000_18250.pkl", "18250_20288.pkl"]

panoptic_anns_json_combined = []

for picklePiece in pickle_list:
    with open(r'C:\Users\andor.neo\Desktop\casey\ml_ai\models-master\research\deeplab\datasets\coco\annotations\annotations_pickles\\' + picklePiece, "rb") as fp:
        sublist = pickle.load(fp)
        panoptic_anns_json_combined.extend(sublist)

len(panoptic_anns_json_combined)

20288

In [38]:
for idx in range(len(panoptic_json["images"])):
    panoptic_json["annotations"][idx]["image_id"] = panoptic_json["images"][idx]["id"]
    panoptic_json["annotations"][idx]["file_name"] = panoptic_json["images"][idx]["file_name"]

In [42]:
for idx in range(len(panoptic_json["images"])):
    panoptic_json["annotations"][idx]["file_name"] = str(panoptic_json["annotations"][idx]["image_id"]).zfill(12) + ".png"

In [134]:
with open(panoptic_json_dir + "panoptic_test-dev2017_mrcnn.json", 'w') as f:
    json.dump(panoptic_json, f)